# Compute Brunt-Vaisala Frequency indicator

In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from acIndUtils import acIndUtils, acIndBruntVaisGraphicUtils

### Loading the perimeter of the area of interest

In [2]:
areaPerimeter = pd.read_csv("areas/areaAdriatic.csv")
verticalLevels = -np.arange(0, 100, 2)
maxDepth = -100 # max depth for vertical profiles

### Reading raw Brunt-Vaisala frequency data 

In [3]:
SrawDataFileSpec = acIndUtils.acNcFileSpec(
    ncFileName="/Users/jalessandri/ADRIACLIM/Indicators/Brunt-Vaisala-Historical/Historical_Nemo/Brunt-Vaisala/AdriaClim_BVF_Hist.nc", 
    varName="BVF", xVarName="lon", yVarName="lat", zVarName="depth", tVarName="time_counter" )
SrawDataFileSpec.printSpec()
SncClippedDataFileSpec = acIndUtils.acCloneFileSpec(SrawDataFileSpec, ncFileName="bvfClippedData.nc")
SannualMapsNcFileSpec = acIndUtils.acCloneFileSpec(SncClippedDataFileSpec, ncFileName="bvfClippedAnnualMean.nc", tVarName="year")
SwinterMapsNcFileSpec = acIndUtils.acCloneFileSpec(SannualMapsNcFileSpec, ncFileName="bvfClippedWinterMean.nc")
SsummerMapsNcFileSpec = acIndUtils.acCloneFileSpec(SannualMapsNcFileSpec, ncFileName="bvfClippedSummerMean.nc")
StrendMapNcSpec = acIndUtils.acCloneFileSpec(SannualMapsNcFileSpec, ncFileName="bvfTrendMap.nc")

  ncFileName: /Users/jalessandri/ADRIACLIM/Indicators/Brunt-Vaisala-Historical/Historical_Nemo/Brunt-Vaisala/AdriaClim_BVF_Hist.nc
  varName: BVF
  xVarName: lon
  yVarName: lat
  zVarName: depth
  tVarName: time_counter


### Generating pre-aggregates

In [ ]:
"""
clipping the input data over the area of interest (may require some time)
"""
clippedData = acIndUtils.acClipDataOnRegion(SrawDataFileSpec, areaPerimeter, SncClippedDataFileSpec.ncFileName)

In [4]:
"""
Generating the temporary nc file with the annual means
"""
acIndUtils.acGenerateAnnualMeanMaps(SncClippedDataFileSpec, SannualMapsNcFileSpec.ncFileName)
print("done")

done


In [ ]:
"""
Generating the temporary nc file with the winter means
"""
acIndUtils.acGenerateSeasonalWinter(SncClippedDataFileSpec, SwinterMapsNcFileSpec.ncFileName)
print("done")

In [ ]:
"""
Generating the temporary nc file with the summer means
"""
acIndUtils.acGenerateSeasonalSummer(SncClippedDataFileSpec, SsummerMapsNcFileSpec.ncFileName)
print("done")

In [5]:
"""
Generating the temporary nc file with the map of trend.
"""
acIndUtils.acComputeSenSlope3DMap(SannualMapsNcFileSpec, StrendMapNcSpec.ncFileName)
print("done")


done


### Visualize Brunt-Vaisala frequency profiles

In [ ]:
"""
Vertical mean profiles of salinity: annual, winter, summer
"""
outvert="/Users/jalessandri/ADRIACLIM/Indicators/Brunt-Vaisala-Historical/BVF_profile_hist_NAdr"

figVProf = acIndBruntVaisGraphicUtils.acBruntVaisPlotVProfileAll(SannualMapsNcFileSpec, SsummerMapsNcFileSpec, SwinterMapsNcFileSpec,\
                                                                 maxDepth, psuLims=(0,13), zlevs=verticalLevels, csv=True, pth=outvert)

In [ ]:
"""
Trend of salinity, vertical profile
"""
outvert="/Users/jalessandri/ADRIACLIM/Indicators/Brunt-Vaisala-Historical/BVF_profile_hist_NAdr"

figVProfTrend = acIndBruntVaisGraphicUtils.acBruntVaisPlotVProfileTrend(StrendMapNcSpec, maxDepth, zlevs=verticalLevels, \
                                                                        psuLims=(-0.35,0.2) , csv=True, pth=outvert) 